This project aims to predict the direction of a stock's price during the last two hours of trading (2 PM - 4 PM), using data from the beginning of the day. 2 PM - 4 PM is the most liquid period, with significant trading activity. Predicting price movements is useful to optimize a portfolio by strategically adjusting the positions.

The three possible directions:
- Sharp price decline
- Minor change (little variation, regardless of direction)
- Sharp price increase

The objective is to use predictive models to anticipate these movements.

# Data

In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb

from scipy.stats import skew, kurtosis, chi2_contingency, kruskal
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score

from sklearn.cluster import KMeans
import sklearn as sk

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
import hdbscan
import os
import optuna


In [ ]:
input_train = pd.read_csv("data/input_training.csv")
output_train = pd.read_csv("data/output_training_gmEd6Zt.csv")

In [ ]:
 # merge X and Y in the same dataframe
data = input_train.merge(output_train, on="ID")

The data :
- Returns. These have a 5-minute granularity. For example, r0: return between the 9:30 and 9:35 prices. Up to r52: return between the 1:55 and 2:00 prices. Returns are calculated in basis points.
- Day: ID of the day corresponding to a stock's intraday returns. Important: no days from the training set are included in the test set to avoid information leaks. We have 502 days, or approximately 2 years of trading data.
- Equity: ID of the stock in question. Similarly, no stock from the training set is found in the test set.

Reod : target variable
- if the yield calculated in basis points between 2 p.m. and 4 p.m. is less than -25 bps: reod takes the value -1
- Between -25 and 25 bps: takes the value 0
- Greater than 25 bps: takes the value 1

# Cleaning and significance testing of features

## Processing missing data

In [ ]:
#number of NA by column
print(data.isna().sum())
# percentage of NAN by column
print(data.isna().mean() * 100)

In [ ]:
print(data.isna().sum(axis=1))

Missing data are only found in the returns. We delete lines where more than 30% of returns are missing. 

In [ ]:
data = data.dropna(axis=0, thresh = data.shape[1] * 0.7)

When a return is missing, we replace it by the last return available before. If the first return is missing, we replace it by the next return.

In [ ]:
# colonnes de rendements
return_cols = [col for col in data.columns if col.startswith('r') and col != 'reod']
data[return_cols] = data[return_cols].bfill(axis=0).ffill(axis=0)

Show the repartition of values in Reod.

In [ ]:
value_counts = data['reod'].value_counts()
value_percentages = (value_counts / len(data)) * 100

for value, count in value_counts.items():
    percentage = value_percentages[value]
    print(f"Value : {value} | Number : {count} | Percentage : {percentage:.2f}%")

The values ​​are roughly evenly distributed.

## Extreme values

We need to check for extreme or even outlier values ​​in the data, specifically in the stocks returns.

We look at how many returns are >100bp or <100bp, which corresponds to an increase/decrease of more than 1% in 5 minutes.

We also look at the minimum and maximum values ​​to see if they are high but economically consistent, or if there may be any outliers.

In [ ]:
# returns
rendement_cols = [col for col in data.columns if col.startswith("r") and col != "reod"]

# put them in a serie
stacked_rendements = (data[["ID"] + rendement_cols].astype({col: "float32" for col in rendement_cols})
                      .set_index("ID").stack().reset_index())
stacked_rendements.columns = ["ID", "colonne", "valeur"]

# we group them according to < 100 ; > 100 or between -100 and 100
rendements = stacked_rendements["valeur"]
groupe_A = rendements[(rendements >= -100) & (rendements <= 100)].count()
groupe_B = rendements[rendements < -100].count()
groupe_C = rendements[rendements > 100].count()
total = len(rendements)

print("Repartition of returns (bps) :")
print(f"Returns between [-100 ; 100] : {groupe_A} observations ({groupe_A/total:.2%})")
print(f"Returns < -100: {groupe_B} observations ({groupe_B/total:.2%})")
print(f"Returns > 100: {groupe_C} observations ({groupe_C/total:.2%})")
print(f"Extreme returns : {(groupe_B + groupe_C)/total:.2%}")

# top 5 of highest and lowest returns
top_5 = stacked_rendements.sort_values("valeur", ascending=False).head(5)
bottom_5 = stacked_rendements.sort_values("valeur", ascending=True).head(5)
print("\n 5 highest returns :")
print(top_5)
print("\n 5 lowest returns :")
print(bottom_5)

The percentage of extreme returns is correct : this could correspond to days of announcements, publications, market stress, etc. However, we have some outlier extreme values ​​that need to be addressed.

We will remove all returns exceeding a variation of more than 15%. This gives us enough margin to still account for large market movements, while remaining within the limits of what is possible.

In [ ]:
#delete extreme returns
data[rendement_cols] = data[rendement_cols].where((data[rendement_cols] >= -1500) & (data[rendement_cols] <= 1500))

# replace NAN by the previous value if it exists
data[rendement_cols] = data[rendement_cols].apply(lambda row: row.ffill(), axis=1)

# for NAN returns without a previous value, we replace them by the next return
data[rendement_cols] = data[rendement_cols].apply(lambda row: row.bfill(), axis=1)

Plot the returns distribution.

In [ ]:
rendements_serie = data[rendement_cols].stack().dropna()
plt.figure(figsize=(10, 5))
plt.hist(rendements_serie, bins=200, edgecolor='black')
plt.title("Distribution of returns (log scale)")
plt.xlabel("Return (bps)")
plt.ylabel("Frequency")
plt.yscale("log")
plt.grid(True)
plt.show()

In [ ]:
mean_val = rendements_serie.mean()
std_val = rendements_serie.std()
skew_val = skew(rendements_serie)
kurt_val = kurtosis(rendements_serie, fisher=False)

print("\Statistical moments of returns (en bps) :")
print(f"Mean: {mean_val:.2f}")
print(f"Standard deviation: {std_val:.2f}")
print(f"Skewness: {skew_val:.4f}")
print(f"Kurtosis: {kurt_val:.2f}")

- The mean is almost null, which means the distribution is almost centered.
- The standard deviation represents reasonable volatility (0.3452% in 5 minutes). This is consistent for such short time horizons.
- The skew is negative, so there is a strong left asymmetry, which means that large declines are more frequent than large increases. This is normal because the market always falls much faster than it rises (panic effect).
- The kurtosis is very positive: the distribution tails are much thicker than those of a normal distribution.

## Correlation between the features and the target variable

- For continuous features (returns), we perform a Krustal-Wallis test to verify whether the distribution of returns varies significantly between the classes of reod.
- For categorical features (equity, day), we perform a Chi² test to verify whether the distribution of the categorical feature is significantly different between reod classes.

### Returns

Test Krustal Wallis

In [ ]:
features = data.columns[3:-1]  #r0 to r52
p_values = {}

for feature in features:
    groups = [data[data['reod'] == cat][feature] for cat in data['reod'].unique()]
    stat, p = kruskal(*groups)
    p_values[feature] = p

# non significant features (p >= 0.05)
non_significant_features = [k for k, v in p_values.items() if v is None or v >= 0.05]

print("Non significant features (p ≥ 0.05) :")
print(non_significant_features)

# sort the most significant features
sorted_features = sorted(p_values.items(), key=lambda x: x[1])
print("10 most correlated features :")
print(sorted_features[:10])

All returns are significant (except two), which mean they influence the target variable.

### Categorical features

In [ ]:
#contingency table (frequency of reod for each day)
contingency_table = pd.crosstab(data['day'], data['reod'])

# Test Chi2
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Stat Chi² : {chi2:.4f}")
print(f"P-value : {p:.8f}")

if p < 0.05:
    print("`day` significantly influences `reod`")
else:
    print("No significant relationship between `day` and `reod`")

According to the Chi-square test, we reject hypothesis H0 with certainty. The equity variable significantly influences the target variable. However, we can note that the p-value is zero, which is strange. This could potentially be explained by the fact that we have a large number of variables, so the Chi-square test is less robust. We will use Cramer's V to confirm the result.

In [ ]:
def cramers_v(x, y):
    contingency_table = pd.crosstab(x, y)
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    r, k = contingency_table.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

# Cramer's V bewteen  day and reod
v_cramer = cramers_v(data['day'], data['reod'])
print(f" V de Cramer entre `day` et `reod` : {v_cramer:.4f}")

# Cramer's V beetween equity and reod
v_cramer_equity = cramers_v(data['equity'], data['reod'])
print(f" V de Cramer entre `equity` et `reod` : {v_cramer_equity:.4f}")

There is a significant correlation between the day and the target variable, and between equity and the target variable.

We can also group the equity variables according to the reod value most associated with them. We then analyze the extent to which the reod variable depends on the equity type to which it belongs.

In [ ]:
# find the most frequent reod for each equity
dominant_reod_per_equity = data.groupby('equity')['reod'].agg(lambda x: x.value_counts().idxmax())
equity_reod_group_series = data['equity'].map(dominant_reod_per_equity)

# Cramer's V between this serie and the true variable 'reod'
v_cramer = cramers_v(equity_reod_group_series, data['reod'])
print(f"Cramer's V after regrouping : {v_cramer:.4f}")


Cramer's V is quite low for the equity variable. This may be explained by the fact that equity is not a "true" categorical variable. Later in the project, we will use unsupervised training to create equity clusters.

# Benchmark


We don't have a detailed description of how the challenge benchmark is calculated. We know that it's based on the creation of features based on returns r0 to r52, that it compares the behavior of different stocks and days, and that it uses a basic classifier based on the state of the tree. They achieved a score of 41.74%. The challenge winner achieved 51.93%.

We test a naive benchmark:
- we look at whether, between 9:30 a.m. and 2:00 p.m., the stock rose 25bp, fell 25bp, or remained stable. We then classify them based on this into +1, -1, and 0.

In [ ]:
benchmark = data.copy()

cols_to_keep = ["ID", "day", "equity", "r0", "r52", 'reod']
benchmark = benchmark[cols_to_keep]
benchmark["reod_estimated"] = benchmark["r52"] - benchmark["r0"]
benchmark["reod_estimated"] = benchmark["reod_estimated"].apply(lambda x: 1 if x > 25 else (-1 if x < -25 else 0))

total_percentage = (((benchmark["reod"] == benchmark["reod_estimated"]).sum()) / len(benchmark)) * 100

print(f"Percentage of identical values : {total_percentage:.2f}%")
benchmark.head()

We find a benchmark slightly lower than the one given in the subject.

# Creating new features

Create multiple features to better analyze the dataset.

## New features

We begin by creating features derived from standard statistics:
- mean, median of returns over a day
- standard deviation and variance of returns over a day
- amplitude (difference between the maximum and minimum returns for the day).

These calculations are applied by equity.

In [ ]:
data['mean_return'] = data.iloc[:, 3:-1].mean(axis=1)
data['median_return'] = data.iloc[:, 3:-1].median(axis=1)
data['std_return'] = data.iloc[:, 3:-1].std(axis=1)
data['var_return'] = data.iloc[:, 3:-1].var(axis=1)
data['range_return'] = data.iloc[:, 3:-1].max(axis=1) - data.iloc[:, 3:-1].min(axis=1)

We then calculate:
- the dynamics of returns: their trend at the start and end of the session, then we calculate the difference between the trends. (for each day)

- the momentum over 10min and 20min. If it is > 0, this means that the stock tends to rise in the last minutes before 2 p.m. (for each day)

- the acceleration, to determine whether the stock tends to pick up speed or slow down at the end of the session. (for each day)

- the volatility ratio. We compare recent volatility to high volatility. If volatility has recently increased, it means that the market is more unstable towards the end of the session. (for each day)

In [ ]:
data['trend_last_5'] = data[['r48', 'r49', 'r50', 'r51', 'r52']].mean(axis=1)
data['trend_first_5'] = data[['r0', 'r1', 'r2', 'r3', 'r4']].mean(axis=1)
data['trend_diff'] = data['trend_last_5'] - data['trend_first_5']

data['momentum_10'] = data['r52'] - data['r42']
data['momentum_20'] = data['r52'] - data['r32']
data['acceleration'] = data['momentum_10'] - data['momentum_20']

#we look at the volatility of the last half hour before 2 p.m., compared to the total volatility of the day
vol_fin = data[['r47', 'r48', 'r49', 'r50', 'r51', 'r52']].std(axis=1)
vol_totale = data['std_return']

#if all the returns from r47 to r52 are 0, and the standard deviation is 0 -> we generate a NAN. we replace with 0
data['volatility_ratio'] = np.where((vol_fin == 0) & (vol_totale == 0), 0.0, vol_fin / vol_totale)

The market has cycles (start/end of month effect, quarter, etc.) that can be captured with a sinusoidal transformation on the day data.

In [ ]:
data['day_sin'] = np.sin(2 * np.pi * data['day'] / max(data['day']))
data['day_cos'] = np.cos(2 * np.pi * data['day'] / max(data['day']))

We can also transform "day" into time categories to capture period effects. We divide the "day" variable into equal quantiles.

In [ ]:
data['day_bin'] = pd.qcut(data['day'], q=10, labels=False)

We now calculate cross-temporal features, i.e., across several days of the sample.

For each equity, we calculate:
- the moving average of its 5-day and 10-day returns
- the rolling volatility over the same intervals
- the momentum over the same intervals (the difference between the average returns of the two days forming the interval).
- the volatility ratio: the ratio between the 5-day rolling volatility and the 10-day rolling volatility

Check if some equities have less than 11 days of observations, because we could not apply a 10-day rolling window on them.

In [ ]:
equity_sizes = data['equity'].value_counts()
equity_to_remove = equity_sizes[equity_sizes < 11]
n_to_drop = equity_to_remove.sum()

print(f"Equity with <11 observations: {len(equity_to_remove)}")
print(f"Number of rows to delete : {n_to_drop}")

There are very few equities that have less than 11 observations, and in total that represents 137 lines which is very few given the dataset, so we can afford to delete them.

In [ ]:
equity_supprimer = equity_sizes[equity_sizes < 11].index
data = data[~data['equity'].isin(equity_supprimer)].reset_index(drop=True)

In [ ]:
# returns columns
return_cols = [col for col in data.columns if col.startswith("r") and col != "reod"]

# sort by equity and day
data = data.sort_values(by=['equity', 'day'])

# features rolling
for equity in data['equity'].unique():
    mask = data['equity'] == equity

    mean_return = data.loc[mask, 'mean_return']
    std_return = data.loc[mask, 'std_return']

    # Moving average
    data.loc[mask, 'Moyenne mobile 5d'] = mean_return.rolling(window=5).mean().shift(1)
    data.loc[mask, 'Moyenne mobile 10d'] = mean_return.rolling(window=10).mean().shift(1)

    # Moving volatility
    vol5 = std_return.rolling(window=5).mean().shift(1)
    vol10 = std_return.rolling(window=10).mean().shift(1)

    data.loc[mask, 'Vol mobile 5d'] = vol5
    data.loc[mask, 'Vol mobile 10d'] = vol10

    # Momentum
    data.loc[mask, 'Momentum mobile 5d'] = (mean_return - mean_return.shift(5)).shift(1)
    data.loc[mask, 'Momentum mobile 10d'] = (mean_return - mean_return.shift(10)).shift(1)

    # Vol ratio 10d
    vol_ratio = np.where((vol5 == 0) & (vol10 == 0), 0.0, #if division 0/0 then = 0
        np.where( vol10 == 0,np.nan, vol5 / vol10)) # if numerator / 0 then = NAN
    data.loc[mask, 'Vol ratio 10d'] = vol_ratio


In [ ]:
data = data.dropna()
data = data.drop(columns=['ID'])

# Construction of features with clustering (unsupervised model)

## Equities cluster on features

Previously, we tried to consider equity stocks as categorical features in order to analyze their impact on the target variable, reod. However, equity stocks are not strictly speaking categorical features. To properly analyze their impact on the target variable, we will create equity clusters to group them according to their common characteristics and see if belonging to a certain equity group has an influence on the target variable.

We will group the equity stocks according to certain previously created features that seem relevant for grouping the stocks and analyzing their impact on reod.

Each cluster represents a type of equity behavior. The goal is to be able to say: "This group of stocks has a specific trading pattern, so when a new equity exhibits this same pattern, I can predict that it will likely exhibit this type of behavior (e.g., a sharp rise at the end of the session)."

We start by doing a PCA on the features that we find interesting for grouping equity.

In [ ]:
# relevant features to group equities
features_clustering = ['mean_return', 'std_return', 'var_return', 'momentum_10', 'momentum_20',
                      'acceleration', 'volatility_ratio', 'day_sin', 'day_cos', 'day_bin',
                      'Moyenne mobile 5d', 'Moyenne mobile 10d', 'Vol mobile 5d', 'Vol mobile 10d',
                       'Momentum mobile 5d', 'Momentum mobile 10d', 'Vol ratio 10d']

# normalize the data in an economically meaningful way
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(data[features_clustering])

# PCA
pca_full = PCA()
X_train_pca = pca_full.fit_transform(X_train_scaled)
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# explained variance and scatter plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].scatter(X_train_pca[:, 0], X_train_pca[:, 1], marker="x")
axes[0].set_title("PCA")
axes[0].set_xlabel('Principal Component 1')
axes[0].set_ylabel('Principal Component 2')

axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title("Cumulative Explained Variance by Component")
axes[1].grid(True)
plt.tight_layout()
plt.show()


90% of the variance can be explained by 7 components.

#### HDBSCAN

Group the equity according to certain features that seem interesting (the same as above) and apply an HDBSCAN model to calculate the optimal number of clusters and how to distribute the equity by cluster.

In [ ]:
# aggregate selected features using: mean, std, 75th percentile and 25th percentile
aggregated = data.groupby('equity')[features_clustering].agg(['mean', 'std', lambda x: x.quantile(0.25),
                                                              lambda x: x.quantile(0.75)])

# rename columns properly
aggregated.columns = [f"{col[0]}_{col[1] if isinstance(col[1], str) else 'q'+str(int(col[1](None)*100))}"
                      for col in aggregated.columns]
std_cols = [col for col in aggregated.columns if '_std' in col]
aggregated[std_cols] = aggregated[std_cols].fillna(0)

# normalization in an economically meaningful way
scaler = RobustScaler()
X_scaled = scaler.fit_transform(aggregated)

# create clusters
clusterer = hdbscan.HDBSCAN(min_cluster_size=10)
aggregated['cluster'] = clusterer.fit_predict(X_scaled)
print("Number of equities per cluster:")
print(aggregated['cluster'].value_counts(dropna=False))


- The vast majority of equities are grouped in cluster 2. Cluster 3 may also be interesting given the number of equities within it.
- According to the HDBSCAN documentation, "any point not in a selected cluster is simply a noise point (and assigned the label -1)." This means that 286 points are not well classified.
- We will verify these results with K-means clustering to be able to interpret them.

Visualization of our clustering: we use a PCA to reduce the dimension of the problem and be able to visualize our clusters in 2D.

In [ ]:
# ACP
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
aggregated['PCA1'] = X_pca[:, 0]
aggregated['PCA2'] = X_pca[:, 1]

# plot clustering
plt.figure(figsize=(8, 4))
sns.scatterplot(data=aggregated, x='PCA1', y='PCA2', hue='cluster', palette='tab10', s=60)
plt.title("Clustering with HDBSCAN")
plt.grid(True)
plt.tight_layout()
plt.show()

- Except for group 0, the clusters overlap, which isn't very conclusive.
- Finally, group 3 isn't necessarily very interesting because it's very close to group 2. It's more likely group 0 that's interesting because it's very separate from the rest, so the clustering works well (we do have a very different trading pattern for these stocks).

#### KMEANS

We cluster following the same logic, but this time using a K means model.

In [ ]:
available_features = [f for f in features_clustering if f in data.columns]
# mean by equity
equity_features = data.groupby("equity")[available_features].mean().reset_index()

# normalize data using the right method
X = equity_features.drop(columns=["equity"])
scaler = RobustScaler()
X_scaled_Kmeans = scaler.fit_transform(X)

# ACP
pca_kmeans = PCA(n_components=9)
X_pca = pca_kmeans.fit_transform(X_scaled_Kmeans)

The optimal number of clusters is checked using the elbow method.

In [ ]:
# elbow method
inertia = []
K = range(1, 11)

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_pca)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K, inertia, marker='o')
plt.xlabel("Number of clusters k")
plt.ylabel("Inertia intra-cluster")
plt.title("Elbow method (KMeans after PCA)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# creation of the clusters
kmeans = KMeans(n_clusters=4, random_state=42)  # number of clusters = 4, given by the elbow method
clusters = kmeans.fit_predict(X_pca)            #clusters on PCA
equity_features['cluster'] = clusters

# visualise
plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=clusters, palette='tab10', s=60)
plt.title("KMeans Clusyers")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.grid(True)
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

In [ ]:
compter_equity = equity_features['cluster'].value_counts().sort_index()
pourcentage_equity = (compter_equity / compter_equity.sum()) * 100
resultat_equity= pd.DataFrame({'Number of equities': compter_equity, "Percentage(%)": pourcentage_equity.round(2)})
print(resultat_equity)

- The vast majority of equities are concentrated in a cluster. This means that the equities in the dataset behave similarly. Economically, this means that the vast majority of stocks in the US market perform similarly, which isn't necessarily surprising.

- The four clusters are still well separated on the graph, which means they clearly explain different trading behaviors. Clustering works better than with HDBSCAN.

Now that we've categorized the equities into clusters, we add this information as a feature to our dataset. We chose to use the clusters created by Kmeans because they are more conclusive than those created by HDBSCAN and seem to provide more information.

In [ ]:
data = data.merge(equity_features[['equity', 'cluster']], on='equity', how='left')

## Equities cluster on returns

We're testing clustering on returns to see if it can provide additional information, wich mean clustering the equity based on their returns, not based on their features. We're using K means.

In [ ]:
rendement_cols = [col for col in data.columns if col.startswith("r")]

# normalize returns
scaler_rendement = RobustScaler()
X_rendement_scaled = scaler_rendement.fit_transform(data[rendement_cols])

# PCA
pca_rendement = PCA()
X_rendement_pca = pca_rendement.fit_transform(X_rendement_scaled)
cumulative_variance_rendement = np.cumsum(pca_rendement.explained_variance_ratio_)

# explained variance and PCA plot
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].scatter(X_rendement_pca[:, 0], X_rendement_pca[:, 1], marker="x")
axes[0].set_title("PCA on returns")
axes[0].set_xlabel('Principal component 1')
axes[0].set_ylabel('Principal component 2')

axes[1].plot(range(1, len(cumulative_variance_rendement) + 1),
             cumulative_variance_rendement, marker='o')
axes[1].set_xlabel('Number of components')
axes[1].set_ylabel('Cumulative explained variance')
axes[1].set_title("Cumulative explained variance by component")
axes[1].grid(True)
plt.tight_layout()
plt.show()


The optimal number of clusters is checked using the elbow method.

In [ ]:
inertia_rendement = []
K_rendement = range(1, 11)
for k in K_rendement:
    kmeans_rendement = KMeans(n_clusters=k, random_state=42)
    kmeans_rendement.fit(X_rendement_pca[:, :2])
    inertia_rendement.append(kmeans_rendement.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K_rendement, inertia_rendement, marker='o')
plt.xlabel("Number of clusters k")
plt.ylabel("Inertia intra-cluster")
plt.title("Elbow method (KMeans on returns PCA)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# KMEANS with 4 clusters (determined by the elbow method)
kmeans_final_rendement = KMeans(n_clusters=4, random_state=42)
clusters_rendement = kmeans_final_rendement.fit_predict(X_rendement_pca[:, :2])

# visualize clusters
plt.figure(figsize=(8, 4))
sns.scatterplot(x=X_rendement_pca[:, 0], y=X_rendement_pca[:, 1], hue=clusters_rendement, palette='tab10', s=60)
plt.title("KMeans Clusters on PCA of Returns")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.grid(True)
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of equities per return cluster
compter_rendement = pd.Series(clusters_rendement).value_counts().sort_index()
pourcentage_rendement = (compter_rendement / compter_rendement.sum()) * 100

# Create result DataFrame
resultat_rendement = pd.DataFrame({'Cluster': compter_rendement.index,
    'Number of equities in cluster': compter_rendement.values,
    'Percentage (%)': pourcentage_rendement.round(2).values})

resultat_rendement = resultat_rendement.set_index("Cluster")

print("\nDistribution of return clusters")
print(resultat_rendement)


- We find 4 clusters, just like when we clustered the equity by their features. It's therefore consistent.
- We add this classification as a feature of our dataframe.

In [ ]:
data['cluster_rendement'] = clusters_rendement

# Normalization, separation of the dataset

- Separate the data to have 80% training and 20% validation (called testing here).
- We ensure that no equity from the test is reflected in the test.

In [ ]:
# group by equity
equity_target = (data.groupby("equity")["reod"]
  .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
  .reset_index())

# split in 80% train + 20%test, with no similar equity between the two sets
equities_train, equities_test = train_test_split(equity_target,
    test_size=0.2, random_state=42, stratify=equity_target["reod"])

train = data[data["equity"].isin(equities_train["equity"])].copy()
test = data[data["equity"].isin(equities_test["equity"])].copy()

- We normalize (in both the train and test) our features and returns, using RobustScaler, which allows us to avoid overly compressing the thick tails of the distribution (so as not to lose the economic information that the extreme values ​​provide us).
- We graphically represent the distribution of returns.

In [ ]:
features_normalisation = ['r0', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8',
       'r9', 'r10', 'r11', 'r12', 'r13', 'r14', 'r15', 'r16', 'r17', 'r18',
       'r19', 'r20', 'r21', 'r22', 'r23', 'r24', 'r25', 'r26', 'r27', 'r28',
       'r29', 'r30', 'r31', 'r32', 'r33', 'r34', 'r35', 'r36', 'r37', 'r38',
       'r39', 'r40', 'r41', 'r42', 'r43', 'r44', 'r45', 'r46', 'r47', 'r48',
       'r49', 'r50', 'r51', 'r52', 'mean_return',
       'median_return', 'std_return', 'var_return', 'range_return',
       'trend_last_5', 'trend_first_5', 'trend_diff', 'momentum_10',
       'momentum_20', 'acceleration', 'volatility_ratio', 'day_sin', 'day_cos',
       'day_bin', 'Moyenne mobile 5d', 'Moyenne mobile 10d', 'Vol mobile 5d',
       'Vol mobile 10d', 'Momentum mobile 5d', 'Momentum mobile 10d','Vol ratio 10d',]
scaler = RobustScaler()
train[features_normalisation] = scaler.fit_transform(train[features_normalisation])
test[features_normalisation] = scaler.transform(test[features_normalisation])

In [ ]:
rendement_cols = [col for col in train.columns if col.startswith("r") and col != "reod"]
rendements_norm = train[rendement_cols].stack().dropna()

plt.figure(figsize=(10, 5))
sns.histplot(rendements_norm, bins=200, kde=False)
plt.yscale('log')  # log scale
plt.title("Distribution of returns")
plt.xlabel("Normalized returns")
plt.ylabel("Frequency (log)")
plt.tight_layout()
plt.show()

The distribution of returns always has thick tails and negative skewness (skewness < 0), meaning that there is a higher frequency of sharp declines than sharp increases. Normalizing does not erase the characteristics of our distribution.

In [ ]:
# Define the target variable (y) and features (X)
train_x = train.drop(columns=['reod'])  # All columns except the target
train_y = train['reod']  # Target

test_x = test.drop(columns=['reod'])  # All columns except the target
test_y = test['reod']  # Target

train_x = train_x.sample(frac=0.1, random_state=42)  # 10% of the data
train_y = train_y.loc[train_x.index]

train_x.fillna(train_x.median(), inplace=True)  # Fill missing values with median


In [ ]:
# Shift classes by `+1` to avoid negative values
train_y = train_y + 1
test_y= test_y + 1

# Training with non supervised model

Out of curiosity, we look at whether the equity clusters we created previously can predict the reod variable well.

In [ ]:
# PCA
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(train_x)

# Clustering on the training set
n_clusters = 7
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters_train = kmeans.fit_predict(X_train_pca)

# Associate to each cluster the mean of the target 'reod'
df_train = pd.DataFrame(X_train_pca, index=train_x.index)
df_train['cluster'] = clusters_train
df_train['reod'] = train_y
map_reod = df_train.groupby('cluster')['reod'].mean().to_dict()

# Apply on the test set
X_test_pca = pca.transform(test_x)
clusters_test = kmeans.predict(X_test_pca)
df_test = pd.DataFrame(X_test_pca, index=test_x.index)
df_test['cluster'] = clusters_test
df_test['reod'] = test_y
df_test['reod_pred_continu'] = df_test['cluster'].map(map_reod)

def to_class(x):
    if x <= 0.5:
        return 0
    elif 1.5 > x > 0.5:
        return 1
    elif x >= 1.5:
        return 2

df_test['reod_pred'] = df_test['reod_pred_continu'].apply(to_class)

# Evaluation
y_true = df_test['reod'].astype(int)
y_pred = df_test['reod_pred'].astype(int)
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy of the 'KMeans + cluster mean' model (test set): {acc:.4f}")
print(df_test[['reod', 'reod_pred', 'cluster']].head())


The accuracy obtained with equity clusters isn't very good. We still keep the feature created through clustering in our dataset, as it might be useful for supervised models.

# Selection of the supervised model

Testing several classification models on the training data to find the one that will maximize the accuracy of the prediction.

In [ ]:
def evaluate_model(model, train_x, train_y, test_x, test_y, cv=5):
    # Train the model
    model.fit(train_x, train_y)

    # Predictions
    y_pred = model.predict(test_x)

    # Performance evaluation (classification)
    accuracy = accuracy_score(test_y, y_pred)
    class_report = classification_report(test_y, y_pred, output_dict=True)

    # Cross-validation with "accuracy" scoring
    cv_scores = sk.model_selection.cross_val_score(model, train_x, train_y, cv=cv, scoring='accuracy')
    mean_cv_acc = cv_scores.mean()  # Mean accuracy from cross-validation

    result = {"Model": model.__class__.__name__,
        "Test Accuracy": accuracy,
        "Mean CV Accuracy": mean_cv_acc,
        "Classification Report": class_report}

    print(f"Accuracy: {accuracy:.4f}, Mean CV Accuracy: {mean_cv_acc:.4f}\n")
    return result


In [ ]:
# List of models to test
models = {
    "XGBoost": XGBClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", solver="saga"),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVC": SVC(kernel="rbf", C=1.0, class_weight="balanced"),
    "Gradient Boosting": GradientBoostingClassifier(),
}

# Test each model
results = {}
for name, model in models.items():
    print(f"Evaluating model: {name}")
    result = evaluate_model(model, train_x, train_y, test_x, test_y)
    results[name] = result


- Xg Boost : Accuracy: 0.4970, Mean CV Accuracy: 0.4979
- Logistic regression : Accuracy: 0.4166, Mean CV Accuracy: 0.4179
- Decision Tree : Accuracy: 0.3882, Mean CV Accuracy: 0.3842
- Random Forest : Accuracy: 0.444, Mean CV Accuracy: 0.4447
- Gradient Boosting : Accuracy: 0.4764, Mean CV Accuracy: 0.4771
- SVC : Accuracy: 0.3246, Mean CV Accuracy: 0.3307

In this 3-class classification task (where a random prediction would have a 33% chance of being correct), the tested models exhibited varied performance.

- XGBoost yielded the best results with an accuracy of 49.7%. This indicates that it successfully captured interesting structure in the data. This score is significantly higher than chance.

- Gradient Boosting (47.6%) and Random Forest (44.4%) also performed quite well, confirming that ensemble models are well-suited to handling complex and potentially noisy data.

- Logistic regression, with 41.7% accuracy, remains adequate but limited, as it relies on linearity assumptions that may not be appropriate for our data.

- The decision tree is not good.

- SVC achieved only 32.5%: it is the lowest-performing model, and especially the longest to train.

The XGBoost model predicts values ​​with the highest accuracy, so we will choose this model for our predictions.

Now, we optimize the hyperparameters of the XGBoost model, using Optuna. Instead of testing randomly (GridSearch), Optuna takes into account previous trials to refine the search.

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10)}

    model = xgb.XGBClassifier( **params,use_label_encoder=False,
        eval_metric='logloss', random_state=42, verbosity=0)

    model.fit(train_x, train_y)
    preds = model.predict(test_x)
    return accuracy_score(test_y, preds)

In [ ]:
# Fixed seed to always get the same results
sampler = optuna.samplers.TPESampler(seed=42)

# Optuna for hyperparameter optimization
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=10)

print("Best parameters:", study.best_params)
print("Best validation accuracy:", study.best_value)

We pass the best parameters found by Optuna to the model.

In [ ]:
# Best parameters found by Optuna
best_params = {
    'n_estimators': 164,
    'learning_rate': 0.14447746112718687,
    'max_depth': 4,
    'min_child_weight': 6,
    'subsample': 0.836965827544817,
    'colsample_bytree': 0.6185801650879991}

# Set seed here as well
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Set seed in the model too
model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(train_x, train_y)

# Predictions of our output variable
y_pred = model.predict(test_x)

# Accuracy calculation
accuracy = accuracy_score(test_y, y_pred)
class_report = classification_report(test_y, y_pred, output_dict=True)

# Cross-validation with "accuracy" scoring
cv_scores = sk.model_selection.cross_val_score(model, train_x, train_y, cv=cv, scoring='accuracy')
mean_cv_acc = cv_scores.mean()  # Average accuracy of cross-validation

result = {
    "Model": model.__class__.__name__,
    "Test Accuracy": accuracy,
    "Mean CV Accuracy": mean_cv_acc,
    "Classification Report": class_report
}

print(f"Accuracy: {accuracy:.4f}, Mean CV Accuracy: {mean_cv_acc:.4f}\n")


We achieved an accuracy of 0.495 on our test dataset.The accuracy is higher than the one from the data challenge benchmark.

Checking if the model is overfitted : we check the accuracy on the training set.

In [ ]:
train_pred = model.predict(train_x)
train_accuracy = accuracy_score(train_y, train_pred)

print("Train accuracy :", train_accuracy)

The accuracy is slightly higher compared to the test, which is a good sign.

However, we have more than 80 features, and we wonder if they are all relevant. We will rerun the model, but select only the 40 best features, to see if this improves the prediction.

In [ ]:
model = xgb.XGBClassifier(**best_params, random_state=42)  # Create a new model with the best parameters
model.fit(train_x, train_y)

importances = pd.Series(model.feature_importances_, index=train_x.columns)
top_k = 40  # Number of features to select
selected_features = importances.sort_values(ascending=False).head(top_k).index

train_x_selected = train_x[selected_features]
test_x_selected = test_x[selected_features]

In [ ]:
models = {    "XGBoost": xgb.XGBClassifier(random_state=42)}
results = {}
for name, model in models.items():
    print(f"{name}")
    result = evaluate_model(model, train_x_selected, train_y, test_x_selected, test_y)
    results[name] = result

On the test dataset, accuracy is improved with this model (0.503 vs. 0.495), although this remains slight. We check again on our training dataset.

In [ ]:
train_pred = model.predict(train_x_selected)
train_accuracy = accuracy_score(train_y, train_pred)

print("Accuracy on Train :", train_accuracy)

We obtain an accuracy of 0.74, which is high and presents a risk of overfitting. This may be due to the fact that we reduced the number of features. We will optimize the hyperparameters to try to recalibrate the model and correct the overfitting.

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10) }

    model = xgb.XGBClassifier(**params,use_label_encoder=False,
        eval_metric='logloss', random_state=42,verbosity=0  )

    model.fit(train_x_selected, train_y)
    preds = model.predict(test_x_selected)
    return accuracy_score(test_y, preds)

In [ ]:
# seed
sampler = optuna.samplers.TPESampler(seed=42)

# optuna for the hyperparameters optimisation
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=10)

print("Best parameters :", study.best_params)
print("Best accuracy on vaidation set :", study.best_value)

In [ ]:
# Best parameters found by Optuna
best_params = {
    'n_estimators': 164,
    'learning_rate': 0.14447746112718687,
    'max_depth': 4,
    'min_child_weight': 6,
    'subsample': 0.836965827544817,
    'colsample_bytree': 0.6185801650879991}

# Seed
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Create the model with the best parameters
model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(train_x_selected, train_y)

# Predictions
y_pred = model.predict(test_x_selected)

# Accuracy computation
accuracy = accuracy_score(test_y, y_pred)
class_report = classification_report(test_y, y_pred, output_dict=True)

# Cross-validation
cv_scores = sk.model_selection.cross_val_score(model, train_x_selected, train_y, cv=cv, scoring='accuracy')
mean_cv_acc = cv_scores.mean()  # Mean accuracy from cross-validation

result = { "Model": model.__class__.__name__,
    "Test Accuracy": accuracy,
    "Mean CV Accuracy": mean_cv_acc,
    "Classification Report": class_report}

print(f"Accuracy: {accuracy:.4f}, Mean CV Accuracy: {mean_cv_acc:.4f}\n")


In [ ]:
train_pred = model.predict(train_x_selected)
train_accuracy = accuracy_score(train_y, train_pred)

print("Accuracy on Train :", train_accuracy)

The new model with XgBoost, applied only to the best features and with optimized hyperparameters, achieves an accuracy of 0.57 on the train set versus 0.502 on the test set.

The accuracy for the test set is significantly higher than the benchmark. Moreover, the accuracy on the train set is very close to the accuracy of the test set, so we were able to address the overfitting issue through hyperparameter calibration. We believe this model is the best we have tested, and this is the one we are retaining.